## Introdução

In [1]:
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
  raise ValueError("A chave da API da OpenAI não foi encontrada. Defina a variável de ambiente OPENAI_API_KEY em um arquivo chamado `.env` na raiz do projeto.")

In [2]:
from agents import Agent, Runner

In [ ]:
agent = Agent(
  name="Agente basico",
  instructions="Voce é um assistente útil. Responda apenas em CAPSLOCK.",
  model="gpt-4o"
)

In [6]:
result = await Runner.run(agent, "Eai, como você esta?")

In [14]:
result.final_output

'ESTOU FUNCIONANDO PERFEITAMENTE! E VOCÊ, COMO ESTÁ?'

## Output personalizado

In [3]:
from pydantic import BaseModel

In [ ]:
class Receita(BaseModel):
  titulo: str
  ingredientes: list[str]
  tempo_preparo: int
  qntd_refeicoes: int

agente_cozinheiro = Agent(
  name='Agente cozinheiro',
  instructions="""
    Voce é um agente que cria receitas. Você receberá o nome de um prato e deverá dar uma resposta
    detalhada a respeito dessa receita. O tempo de preparo deve ser em minutos. 
  """,
  output_type=Receita
)

In [27]:
response = await Runner.run(agente_cozinheiro, "Strogonoff")
print(response.final_output)

titulo='Strogonoff de Frango' ingredientes=['500g de peito de frango cortado em cubos', '1 cebola média picada', '2 dentes de alho picados', '2 colheres de sopa de óleo ou azeite', '200g de creme de leite', '2 colheres de sopa de ketchup', '1 colher de sopa de mostarda', 'Sal a gosto', 'Pimenta-do-reino a gosto', '100g de champignon fatiado (opcional)', '1 colher de sopa de molho inglês', 'Salsinha picada para decorar'] tempo_preparo=40 qntd_refeicoes=4


## Tool Calling

In [4]:
from agents import function_tool

In [ ]:
@function_tool
def pega_o_tempo(cidade: str) -> str:
  print(f"Tempo na cidade {cidade} é 30°C.")
  return "30°C"

agent = Agent(
  name = "Agente do clima",
  instructions = "Você é um agente que sabem muito sobre climas. Dado uma cidade você deve retorna o clima daquela cidade.",
  model="gpt-4o",
  tools=[pega_o_tempo]
)

In [ ]:
resposta = await Runner.run(agent, "qual o clima na cidade de São Paulo?")
print(resposta.final_output)

Tempo na cidade São Paulo é 30°C.
Em São Paulo, a temperatura atual é de 30°C.


### Handoff

In [ ]:
gerador_tutorial = Agent(
  name="Gerador de tutorial",
  handoff_description="Usado para criar um tutorial baseado em um esboço", #Util para os agentes que chamam esse saber oque ele faz
  instructions=(
    "Dado um tópico de programação e um esboço, sua tarefa é gerar trechos de código para cada seção do esboço."
    "Formate o tutorial em Markdown usando uma mistura de texto para explicação e trechos de código como exemplos."
    "Onde fizer sentido, inclua comentários nos trechos de código para explicar melhor o código."
  )
)

gerador_esboco = Agent(
  name="Gerador de esboço",
  instructions=(
    "Dado um tópico específico de programação, sua tarefa é ajudar a criar um tutorial. Você fará isso elaborando um esboço."
    "Depois de criar o esboço, entregue-o ao agente gerador de tutorial."
  ),
  handoffs=[gerador_tutorial]
)

In [ ]:
resposta = await Runner.run(gerador_esboco, 'variaveis de python')
#print(resposta.final_output)

In [11]:
from agents import handoff, RunContextWrapper

In [12]:
gerador_tutorial = Agent(
  name="Gerador de tutorial",
  handoff_description="Usado para criar um tutorial baseado em um esboço", #Util para os agentes que chamam esse saber oque ele faz
  instructions=(
    "Dado um tópico de programação e um esboço, sua tarefa é gerar trechos de código para cada seção do esboço."
    "Formate o tutorial em Markdown usando uma mistura de texto para explicação e trechos de código como exemplos."
    "Onde fizer sentido, inclua comentários nos trechos de código para explicar melhor o código."
  )
)

def handoff_handler(ctx: RunContextWrapper[None]):
  print("handoff acontecendo")
  return

gerador_esboco = Agent(
  name="Gerador de esboço",
  instructions=(
    "Dado um tópico específico de programação, sua tarefa é ajudar a criar um tutorial. Você fará isso elaborando um esboço."
    "Depois de criar o esboço, entregue-o ao agente gerador de tutorial."
  ),
  handoffs=[handoff(gerador_tutorial, on_handoff=handoff_handler)]
)

In [13]:
resposta = await Runner.run(gerador_esboco, 'variaveis de python')

handoff acontecendo


## Streaming

In [14]:
from openai.types.responses import ResponseTextDeltaEvent

agent = Agent(
  name="Agente de piadas",
  instructions="Você é um agente ajudante que cria piadas quando a respeito de um tópico fornecido."
)

result = Runner.run_streamed(agent, input="Melancia")

async for event in result.stream_events():
  if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
    print(event.data.delta, end="", flush=True)

Por que a melancia foi à festa sozinha?

Porque ela já estava cheia de "sem semente"! 🍉😂

In [ ]:
print("== Run inicia")
async for event in result.stream_events():
  #ignora os deltas (partes da resposta), quero ver apenas o stack trace do agente
  if event.type == "raw_response_event":
    continue
  elif event.type == "agent_updated_stream_event":
    print(f"Agente atualizado: {event.new_agent.name}")
  elif event.type == "run_item_stream_event":
    if event.item.type == "tool_call_item":
      print(f"-- Ferramenta chamada")
    elif  event.item.type == "tool_call_output_item":
      print(f"-- Saída da ferramenta: {event.item.output}")
    elif  event.item.type == "message_output_item":
      print(f"-- Mensagem de saída:\n 
      {ItemHelpers.text_message_output(event.item)}")
    else:
      pass
print("== Run acabou")

## GuardRails